In [216]:
import pandas  as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import NearestNeighbors
DATA = '/kaggle/input/nepali-meals-dataset/meals.csv'
import numpy as np


In [217]:
feature = pd.read_csv(DATA)
X = feature.iloc[:, [1,2,3,6]].values
Y = feature.iloc[:, :1].values


In [218]:
feature.shape

(144, 8)

In [5]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(X_train)

**Training the Date**

In [220]:
model = NearestNeighbors(n_neighbors = 25, algorithm ='brute', metric = 'euclidean')
model.fit(X)

NearestNeighbors(algorithm='brute', metric='euclidean', n_neighbors=25)

In [221]:
new = [1,0,0,0,0,0,0]
input = new[0:3]
input.append(new[5])
input = [input]
dis, indices = model.kneighbors(input)

for value in indices[0]:
  print(Y[value])

['Shrimp Momo']
['Chhurpi']
['Buff Momos']
['Chow Mein']
['Broccoli Momo']
['Chilka Moong Daal (Split Mung Bean)']
['Chapati']
['Carrot Pudding']
['Cauliflower Momo']
['Cashew Pudding']
['Avocado Momo']
['Corn Momo']
['Bhakka']
['Baji']
['Potato Pudding']
['Chickpea Flour Pudding']
['Chicken Choyla']
['Chiura']
['Dal Bhat Tarkari']
['Paratha']
['Chana Masala']
['Pumpkin Momo']
['Sel Roti']
['Semolina Pudding']
['Sel Puri']


# Saving Model

In [222]:
import pickle
file = 'model.pk'

with open(file, 'wb') as f:
  pickle.dump(model, f)

f.close()

# Prediction 

In [223]:
#Prediction
data = '/kaggle/input/nepali-meals-dataset/meals.csv'
df = pd.read_csv(data)

#u_input -> 2D
#user_input-> 1D
#input-> only features to model 

def recommend_meal(user_input):
    saved_model = 'model.pk'
    meals = df.iloc[:, :1].values
    
    recommendation = list()
    
    dis_list = []
    indices_list = []
    
    with open(saved_model, 'rb') as f :
        loaded_model = pickle.load(f)
    
    for u_input in user_input:
        input = u_input[0:3]
        input.append(u_input[5])
        input = [input]
        
        dis, indices = loaded_model.kneighbors(input)
        dis_list.append(dis)
        indices_list.append(indices)
        
        

    combined_dis = np.concatenate(dis_list)
    combined_indices = np.concatenate(indices_list)
    combined_dis= combined_dis.flatten()
    combined_indices= combined_indices.flatten()
    sorted_indices = combined_indices[np.argsort(combined_dis)]
    
#      return sorted_indices
#     return combined_dis, combined_indices
# #     

    
    comb_indices = np.unique(np.array(sorted_indices))
    
    for value in comb_indices:
          recommendation.append(meals[value])
            
    r = np.array(recommendation)
    r = r.flatten()

#     #######
    recommendations = df.loc[df['Nepali Food Name'].isin(r)]

#     #Using the filters
    user_input = u_input
    if(user_input[3]==0.5):
        r1 = recommendations
    else:
        r1 = recommendations[recommendations['Vegetarian']==user_input[3]]

    if (user_input[4]==0.5):
        r2=r1
    else:
        r2 = r1[r1['Dairy-Free']==user_input[4]]

    if (user_input[6]==0.5):
        r3=r2
    else:
        r3 = r2[r2['Oil-Free']==user_input[6]]

    return r3
      
    

In [224]:
import itertools

import itertools

def create_combinations(array):
    if None not in array:
        return [array]

    combinations = []
    none_indices = [i for i, x in enumerate(array) if x == None]
    for combo in itertools.product([0, 0.25, 0.5, 0.75, 1], repeat=len(none_indices)):
        new_array = list(array)
        for i, value in zip(none_indices, combo):
            new_array[i] = value
        combinations.append(new_array)

    return combinations



In [225]:
array = [None, None, 0, 1, 0 ,1,1]
user_inputs = create_combinations(array)

In [226]:
meal = recommend_meal(user_inputs)


In [227]:
meal

,Nepali Food Name,Gluten-Free,Sugar-Free,Keto,Vegetarian,Dairy-Free,Paleo,Oil-Free
50,Kwati,0.1,0.2,0.2,1,0,0.2,1
55,Masoor Daal (Red Lentil),0.1,0.2,0.2,1,0,0.2,1
88,Tandoori Roti,0.0,0.0,0.2,1,0,0.2,1
107,Aloo Gobi,0.2,0.1,0.2,1,0,0.2,1
110,Chana Chaat,0.1,0.3,0.3,1,0,0.3,1
111,Dahi Bhalla,0.2,0.3,0.3,1,0,0.3,1
112,Aloo Tikki,0.3,0.3,0.3,1,0,0.3,1
